Art. 29 Os sistemas e soluções alternativas coletivas de abastecimento de água que utilizam mananciais superficiais devem realizar monitoramento mensal de Escherichia coli no(s) ponto(s) de captação de água.

§ 1º Quando for identificada média geométrica móvel dos últimos 12 meses de monitoramento maior ou igual a 1.000 Escherichia coli/100mL, deve-se avaliar a eficiência de remoção da Estação de Tratamento de Água (ETA) por meio do monitoramento semanal de esporos de bactérias aeróbias.

In [69]:
import os
import re
import sys
import pprint
import pandas as pd
from scipy.stats import gmean
from dateutil.relativedelta import relativedelta

In [70]:
from paths import *

In [71]:
# Parameters
cod_ibge = '3548906' # São Carlos
cod_ibge = '3526902' # Limeira
cod_ibge = '3501608' # Americana

In [26]:
# Read Table
df_bruta = pd.read_excel(
    os.path.join(output_path, str(cod_ibge), 'dados brutos', 'controle', 'controle_mensal_parametros_basicos.xlsx')
)

In [28]:
# Filtra Apenas SAAs
df = df_bruta.loc[df_bruta['Tipo Da Forma De Abastecimento'] == 'SAA']

# Filtra Apenas Último Ano
df = df[df['Ano De Referência'] == max(df['Ano De Referência'])].copy()

['Região Geográfica',
 'Regional De Saúde',
 'Município',
 'Tipo Da Instituição',
 'Sigla Da Instituição',
 'Nome Da Instituição',
 'Cnpj Da Instituição',
 'Nome Do Escritório Regional/Local',
 'Cnpj Do Escritório Regional/Local',
 'Tipo Da Forma De Abastecimento',
 'Código Forma De Abastecimento',
 'Nome Da Forma De Abastecimento',
 'Nome Da Eta / Uta',
 'Tipo De Filtração',
 'Ano De Referência',
 'Mês De Referência',
 'Ponto De Monitoramento',
 'Parâmetro',
 'Campo',
 'Valor',
 'Uf',
 'Código Ibge']

In [ ]:
#df.info()
list(df.columns)

<br>

# Artigo 29

In [29]:
set(df['Parâmetro'])

{'Bactérias Heterotróficas (UFC/mL)',
 'Cloro Residual Livre (mg/L)',
 'Coliformes totais',
 'Cor (uH)',
 'Escherichia coli',
 'Fluoreto (mg/L)',
 'Turbidez (uT)',
 'pH'}

In [30]:
df = df[df['Parâmetro'] == 'Escherichia coli'].copy()
df.head()

,Região Geográfica,Regional De Saúde,Município,Tipo Da Instituição,Sigla Da Instituição,Nome Da Instituição,Cnpj Da Instituição,Nome Do Escritório Regional/Local,Cnpj Do Escritório Regional/Local,Tipo Da Forma De Abastecimento,...,Nome Da Eta / Uta,Tipo De Filtração,Ano De Referência,Mês De Referência,Ponto De Monitoramento,Parâmetro,Campo,Valor,Uf,Código Ibge
327589,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,9,SAÍDA DO TRATAMENTO,Escherichia coli,Número de amostras analisadas,40,SP,350160
327590,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,9,SAÍDA DO TRATAMENTO,Escherichia coli,N de amostras com presença para Escherichia coli,0,SP,350160
327591,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,9,SAÍDA DO TRATAMENTO,Escherichia coli,N de amostras com ausência para Escherichia coli,40,SP,350160
327610,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,2,SAÍDA DO TRATAMENTO,Escherichia coli,Número de amostras analisadas,40,SP,350160
327611,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,2,SAÍDA DO TRATAMENTO,Escherichia coli,N de amostras com presença para Escherichia coli,0,SP,350160


In [31]:
set(df['Ponto De Monitoramento'])

{'SAÍDA DO TRATAMENTO', 'SISTEMA DE DISTRIBUIÇÃO'}

In [32]:
df = df[df['Ponto De Monitoramento'] == 'SAÍDA DO TRATAMENTO'].copy()
df.head()

,Região Geográfica,Regional De Saúde,Município,Tipo Da Instituição,Sigla Da Instituição,Nome Da Instituição,Cnpj Da Instituição,Nome Do Escritório Regional/Local,Cnpj Do Escritório Regional/Local,Tipo Da Forma De Abastecimento,...,Nome Da Eta / Uta,Tipo De Filtração,Ano De Referência,Mês De Referência,Ponto De Monitoramento,Parâmetro,Campo,Valor,Uf,Código Ibge
327589,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,9,SAÍDA DO TRATAMENTO,Escherichia coli,Número de amostras analisadas,40,SP,350160
327590,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,9,SAÍDA DO TRATAMENTO,Escherichia coli,N de amostras com presença para Escherichia coli,0,SP,350160
327591,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,9,SAÍDA DO TRATAMENTO,Escherichia coli,N de amostras com ausência para Escherichia coli,40,SP,350160
327610,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,2,SAÍDA DO TRATAMENTO,Escherichia coli,Número de amostras analisadas,40,SP,350160
327611,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,2,SAÍDA DO TRATAMENTO,Escherichia coli,N de amostras com presença para Escherichia coli,0,SP,350160


In [35]:
df = df[['Ano De Referência', 'Mês De Referência', 'Campo', 'Valor']].copy()
df = df.sort_values(by=['Ano De Referência', 'Mês De Referência', 'Campo']).copy()
df.head()

,Ano De Referência,Mês De Referência,Campo,Valor
327824,2021,1,N de amostras com ausência para Escherichia coli,40
327823,2021,1,N de amostras com presença para Escherichia coli,0
327822,2021,1,Número de amostras analisadas,40
327612,2021,2,N de amostras com ausência para Escherichia coli,40
327611,2021,2,N de amostras com presença para Escherichia coli,0


Americana não tinha amostras no Ponto de captação....

{'SAÍDA DO TRATAMENTO', 'SISTEMA DE DISTRIBUIÇÃO'}

<br>

# Agora vai!

31.03.2022

Descobri mais dados de E.c oli na tabela "demais paramêtros"

In [36]:
# Read Table
df_bruta = pd.read_excel(
    os.path.join(output_path, str(cod_ibge), 'dados brutos', 'controle', 'controle_mensal_demais_parametros.xlsx')
)

In [ ]:
# Filtra Apenas SAAs
df = df_bruta.loc[df_bruta['Tipo Da Forma De Abastecimento'] == 'SAA']

In [37]:
# Filtra Apenas Último Ano
df = df[df['Ano De Referência'] == max(df['Ano De Referência'])].copy()

['Região Geográfica',
 'Regional De Saúde',
 'Município',
 'Tipo Da Instituição',
 'Sigla Da Instituição',
 'Nome Da Instituição',
 'Cnpj Da Instituição',
 'Nome Do Escritório Regional/Local',
 'Cnpj Do Escritório Regional/Local',
 'Tipo Da Forma De Abastecimento',
 'Código Forma De Abastecimento',
 'Nome Da Forma De Abastecimento',
 'Nome Da Eta / Uta',
 'Ano De Referência',
 'Mês De Referência',
 'Data De Registro',
 'Data De Preenchimento Do Relatório Mensal',
 'Data Da Coleta',
 'Tipo De Captação',
 'Categoria Do Manancial Superficial',
 'Nome Do Manancial Superficial',
 'Categoria Do Ponto De Captação Subterrânea',
 'Nome Do Ponto De Captação Subterrânea',
 'Parâmetro',
 'Unidade',
 'Resultado',
 'Uf',
 'Código Ibge']

In [41]:
df.info()
list(df.columns)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 301 to 395
Data columns (total 28 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Região Geográfica                           12 non-null     object 
 1   Regional De Saúde                           12 non-null     object 
 2   Município                                   12 non-null     object 
 3   Tipo Da Instituição                         12 non-null     object 
 4   Sigla Da Instituição                        0 non-null      float64
 5   Nome Da Instituição                         12 non-null     object 
 6   Cnpj Da Instituição                         12 non-null     float64
 7   Nome Do Escritório Regional/Local           0 non-null      float64
 8   Cnpj Do Escritório Regional/Local           0 non-null      float64
 9   Tipo Da Forma De Abastecimento              12 non-null     object 
 10  Código Forma 

['Região Geográfica',
 'Regional De Saúde',
 'Município',
 'Tipo Da Instituição',
 'Sigla Da Instituição',
 'Nome Da Instituição',
 'Cnpj Da Instituição',
 'Nome Do Escritório Regional/Local',
 'Cnpj Do Escritório Regional/Local',
 'Tipo Da Forma De Abastecimento',
 'Código Forma De Abastecimento',
 'Nome Da Forma De Abastecimento',
 'Nome Da Eta / Uta',
 'Ano De Referência',
 'Mês De Referência',
 'Data De Registro',
 'Data De Preenchimento Do Relatório Mensal',
 'Data Da Coleta',
 'Tipo De Captação',
 'Categoria Do Manancial Superficial',
 'Nome Do Manancial Superficial',
 'Categoria Do Ponto De Captação Subterrânea',
 'Nome Do Ponto De Captação Subterrânea',
 'Parâmetro',
 'Unidade',
 'Resultado',
 'Uf',
 'Código Ibge']

In [47]:
df[['Tipo Da Forma De Abastecimento', 'Código Forma De Abastecimento']].nunique()

Tipo Da Forma De Abastecimento    1
Código Forma De Abastecimento     1
dtype: int64

In [49]:
set(df['Parâmetro'])

{'Escherichia coli'}

In [50]:
df = df[df['Parâmetro'] == 'Escherichia coli'].copy()
df.head()

,Região Geográfica,Regional De Saúde,Município,Tipo Da Instituição,Sigla Da Instituição,Nome Da Instituição,Cnpj Da Instituição,Nome Do Escritório Regional/Local,Cnpj Do Escritório Regional/Local,Tipo Da Forma De Abastecimento,...,Tipo De Captação,Categoria Do Manancial Superficial,Nome Do Manancial Superficial,Categoria Do Ponto De Captação Subterrânea,Nome Do Ponto De Captação Subterrânea,Parâmetro,Unidade,Resultado,Uf,Código Ibge
301,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Serviço Municipal e outros,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,SUPERFICIAL,rio,RIO PIRACICABA,NaN,NaN,Escherichia coli,E.coli/100mL,1700,SP,350160
316,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Serviço Municipal e outros,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,SUPERFICIAL,rio,RIO PIRACICABA,NaN,NaN,Escherichia coli,E.coli/100mL,5000,SP,350160
326,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Serviço Municipal e outros,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,SUPERFICIAL,rio,RIO PIRACICABA,NaN,NaN,Escherichia coli,E.coli/100mL,1700,SP,350160
333,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Serviço Municipal e outros,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,SUPERFICIAL,rio,RIO PIRACICABA,NaN,NaN,Escherichia coli,E.coli/100mL,2700,SP,350160
342,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Serviço Municipal e outros,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,SUPERFICIAL,rio,RIO PIRACICABA,NaN,NaN,Escherichia coli,E.coli/100mL,2300,SP,350160


In [72]:
df = df[['Data Da Coleta', 'Parâmetro', 'Unidade', 'Resultado']].copy()
df['Data Da Coleta'] = pd.to_datetime(df['Data Da Coleta'])
df

,Data Da Coleta,Parâmetro,Unidade,Resultado
301,2021-02-19,Escherichia coli,E.coli/100mL,1700.0
316,2021-02-26,Escherichia coli,E.coli/100mL,5000.0
326,2021-03-31,Escherichia coli,E.coli/100mL,1700.0
333,2021-03-03,Escherichia coli,E.coli/100mL,2700.0
342,2021-01-06,Escherichia coli,E.coli/100mL,2300.0
343,2021-01-27,Escherichia coli,E.coli/100mL,700.0
351,2021-01-13,Escherichia coli,E.coli/100mL,1700.0
356,2021-03-10,Escherichia coli,E.coli/100mL,1700.0
357,2021-03-24,Escherichia coli,E.coli/100mL,1400.0
364,2021-01-20,Escherichia coli,E.coli/100mL,2700.0


In [73]:
# ddd
ultima_amostra = max(df['Data Da Coleta'])
um_ano_antes = ultima_amostra - relativedelta(years=1)

print('Ultima Amostra foi {}\nUm ano antes{}'.format(ultima_amostra, um_ano_antes))

Ultima Amostra foi 2021-03-31 00:00:00
Um ano antes2020-03-31 00:00:00


In [74]:
df['Resultado'] = df['Resultado'].astype(str).str.replace(',','.')
df['Resultado'] = df['Resultado'].astype(float).fillna(0.0)

# ddd
df_gmean = df[df['Data Da Coleta'] >= um_ano_antes].copy()
round(gmean(df_gmean['Resultado']), 2)

2059.79